# Chapter 1: Private Keys, Public Keys, and Address Encoding

> **Reference**: `book/manuscript/Chapter 1 Private Keys Public Keys and Address Encoding.md`  
> **Code Examples**: `code/chapter01/`  
> **Last Updated**: 2025-12-05

---

Understanding Bitcoin's cryptographic foundation is essential before diving into Taproot's advanced features. This chapter covers the fundamentals of private keys, public keys, and address generation—the building blocks that make Bitcoin transactions possible.

## The Cryptographic Hierarchy

Bitcoin's security model relies on a one-way mathematical relationship between private keys, public keys, and addresses:

```
Private Key (256-bit) → Public Key (ECDSA point) → Address (encoded hash)
```

Each step in this hierarchy serves a specific purpose:

- **Private keys** provide cryptographic ownership and signing capability
- **Public keys** enable signature verification and payment authorization
- **Addresses** offer a user-friendly way to receive payments while preserving privacy

## Private Keys: The Foundation of Ownership

A Bitcoin private key is fundamentally a 256-bit random number—a massive integer that serves as the secret foundation of cryptocurrency ownership. To put this in perspective, the total number of possible private keys (2^256) exceeds the estimated number of atoms in the observable universe.

### Generating Private Keys

Let's start with a practical example using Python's `bitcoin-utils`:

In [4]:
# Example 1: Generating Private Keys
# Reference: code/chapter01/01_generate_private_key.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey

# Setup mainnet (or 'testnet' for test network)
setup('mainnet')

# Generate a new Bitcoin private key
priv = PrivateKey()

# Extract the private key in different formats
private_key_hex = priv.to_bytes().hex()  # 32 bytes (256-bit) in hexadecimal
private_key_wif = priv.to_wif()          # Wallet Import Format

print(f"Private Key (HEX): {private_key_hex}")
print(f"Private Key (WIF): {private_key_wif}")

Private Key (HEX): 2bb645ed2e8a09ef98aaf002e562f7125cf4846eca80b57d512c94b203d69599
Private Key (WIF): KxggUAdP6Un9LVNzVzNueG64KnHepxEq2nnvfqyWFqZNSccZyEs8


**Note:** Each time you run this code, a new random private key will be generated. The output format is:
- **HEX**: 64-character hexadecimal string (32 bytes)
- **WIF**: Base58Check encoded string, mainnet keys start with `L` or `K`

The hexadecimal representation contains exactly 64 characters (each representing 4 bits), totaling 256 bits or 32 bytes. This format is mathematically precise but not human-friendly for storage or import/export operations.

### Wallet Import Format (WIF)

The Wallet Import Format (WIF) addresses the usability challenges of raw hexadecimal private keys by applying Base58Check encoding. This encoding:

- Adds error detection through checksums
- Eliminates visually confusing characters (0, O, I, l)
- Provides a standardized format for wallet import/export

The WIF encoding process follows these steps:

1. **Add version prefix**: `0x80` for mainnet, `0xEF` for testnet
2. **(Optional) Add compression flag**: If the corresponding public key will be compressed, append 0x01 to the payload. This step changes the final Base58 prefix of the WIF
3. **Calculate checksum**: Apply SHA256(SHA256(data)) and take first 4 bytes
4. **Apply Base58 encoding**: Convert to human-readable format

The resulting WIF strings have distinctive prefixes:

- **L** or **K**: Mainnet private keys
- **c**: Testnet private keys

## Public Keys: Cryptographic Verification Points

Public keys in Bitcoin are points on the secp256k1 elliptic curve, derived from private keys through elliptic curve multiplication. While the mathematical details involve complex curve arithmetic, the practical implementation is straightforward.

### ECDSA and secp256k1

Bitcoin uses the secp256k1 curve for its elliptic curve digital signature algorithm (ECDSA). The secp256k1 curve is defined by the equation:

```
y² = x³ + 7
```

Without diving into the mathematical complexities, understand that:

- Each private key `k` generates a unique point `(x, y)` on the curve
- This relationship is computationally irreversible
- The curve's properties ensure cryptographic security

### Compressed vs Uncompressed Public Keys

Public keys can be represented in two formats:

**Uncompressed format (65 bytes):**

```
04 + x-coordinate (32 bytes) + y-coordinate (32 bytes)
```

**Compressed format (33 bytes):**

```
02/03 + x-coordinate (32 bytes)
```

Compression works because the elliptic curve's mathematical properties allow reconstructing the y-coordinate from the x-coordinate, given only the parity (even/odd) of y:

- `02` prefix: y-coordinate is even
- `03` prefix: y-coordinate is odd

In [2]:
# Example 2: Generating Public Keys
# Reference: code/chapter01/02_generate_public_key.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey

# Setup mainnet (or 'testnet' for test network)
setup('mainnet')

# Generate a new Bitcoin private key
priv = PrivateKey()

# Get the public key (compressed by default)
pub = priv.get_public_key()

# Generate public keys in both formats
public_key_compressed = pub.to_hex(compressed=True)    # 33 bytes
public_key_uncompressed = pub.to_hex(compressed=False)  # 65 bytes

print(f"Compressed:   {public_key_compressed}")
print(f"Uncompressed: {public_key_uncompressed[:70]}...") 
# Truncated for display

Compressed:   020c27ad0b162c64843f3662cb619ebd876f6dcf6bc93be1653b7311b96864a8a7
Uncompressed: 040c27ad0b162c64843f3662cb619ebd876f6dcf6bc93be1653b7311b96864a8a7a5e3...


Modern Bitcoin implementations use only compressed public keys because they are smaller and equally secure.

### X-Only Public Keys: Taproot's Innovation

Taproot introduces **x-only public keys**, which use only the x-coordinate without the y-coordinate parity information. This 32-byte format:

- Reduces transaction size
- Simplifies signature verification
- Enables key aggregation techniques

In [ ]:
# Example 3: Taproot X-Only Public Keys
# Reference: code/chapter01/03_taproot_xonly_pubkey.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey

# Setup mainnet (or 'testnet' for test network)
setup('mainnet')

# Generate a new Bitcoin private key
priv = PrivateKey()

# Get the public key
pub = priv.get_public_key()

# Taproot uses x-only public keys (32 bytes)
# Get the x-coordinate only
taproot_pubkey = pub.to_x_only_hex()  # 32 bytes, x-coordinate only
print(f"X-only Public Key: {taproot_pubkey}")

This innovation plays a crucial role in Taproot's efficiency improvements, which we will explore in detail in later chapters.

## Address Generation: From Public Keys to Payment Destinations

Bitcoin addresses are **not** public keys—they are encoded hashes of public keys. This additional layer provides:

- **Privacy**: Addresses don't directly expose public keys
- **Post-quantum resistance**: Hash functions provide post-quantum security
- **Error detection**: Encoding includes checksums

### Address Generation Process

All Bitcoin addresses follow a similar pattern:

1. **Hash the public key**: Apply SHA256, then RIPEMD160 (or Hash160)
2. **Add metadata**: Version byte and script type information
3. **Add checksum**: Error detection mechanism
4. **Encode**: Base58Check or Bech32 encoding

In [ ]:
# Example 4: Generating Different Address Types
# Reference: code/chapter01/04_generate_addresses.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.keys import P2shAddress, P2wpkhAddress

# Setup mainnet (or 'testnet' for test network)
setup('mainnet')

# Generate a new Bitcoin private key
priv = PrivateKey()

# Get the public key
pub = priv.get_public_key()

# Generate different address types from the same key
legacy_address = pub.get_address()                    # P2PKH
segwit_native = pub.get_segwit_address()              # P2WPKH
taproot_address = pub.get_taproot_address()          # P2TR

# For P2SH-P2WPKH, we need to wrap the P2WPKH script in a P2SH
segwit_script = segwit_native.to_script_pub_key()
segwit_p2sh = P2shAddress.from_script(segwit_script)  # P2SH-P2WPKH

print(f"Legacy (P2PKH):     {legacy_address.to_string()}")
print(f"SegWit Native:      {segwit_native.to_string()}")
print(f"SegWit P2SH:        {segwit_p2sh.to_string()}")
print(f"Taproot:            {taproot_address.to_string()}")

## Address Types and Encoding Formats

### Base58Check Encoding

Base58Check encoding, used for legacy addresses, eliminates visually similar characters and includes error detection:

**Excluded characters:** `0` (zero), `O` (capital o), `I` (capital i), `l` (lowercase L)

**P2PKH (Pay-to-Public-Key-Hash):**

- Prefix: `1`
- Format: Base58Check encoded
- Usage: Original Bitcoin address format
- Example: `1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa`

**P2SH (Pay-to-Script-Hash):**

- Prefix: `3`
- Format: Base58Check encoded
- Usage: Multi-signature and wrapped SegWit addresses
- Example: `3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy`

### Bech32 Encoding: SegWit's Innovation

Bech32 encoding was introduced with SegWit, providing better error detection and correction:

**P2WPKH (Pay-to-Witness-Public-Key-Hash):**

- Prefix: `bc1q`
- Format: Bech32 encoded
- Advantages: Lower fees, better error detection
- Example: `bc1qw508d6qejxtdg4y5r3zarvary0c5xw7kygt080`

### Bech32m Encoding: Taproot's Enhancement

Taproot addresses use Bech32m, an improved version of Bech32:

**P2TR (Pay-to-Taproot):**

- Prefix: `bc1p`
- Format: Bech32m encoded
- Advantages: Enhanced privacy, script flexibility
- Example: `bc1plz0h3rlj2zvn88pgywqtr9k3df3p75p3ltuxh0`

## Address Format Comparison

| Address Type | Encoding | Data Size | Address Length | Prefix | Primary Use Case |
| --- | --- | --- | --- | --- | --- |
| **P2PKH** | Base58Check | 25 bytes | ~34 chars | `1...` | Legacy payments |
| **P2SH** | Base58Check | 25 bytes | ~34 chars | `3...` | Multi-sig, wrapped SegWit |
| **P2WPKH** | Bech32 | 21 bytes | 42-46 chars | `bc1q...` | SegWit payments |
| **P2TR** | Bech32m | 33 bytes | 58-62 chars | `bc1p...` | Taproot payments |

In [ ]:
# Example 5: Verify Address Formats and Sizes
# Reference: code/chapter01/05_verify_addresses.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.keys import P2shAddress, P2wpkhAddress
import base58

def verify_address(address_obj, address_str, address_type):
    """Verify address format and extract information"""
    print(f"\n{address_type}:")
    print(f"  Address: {address_str}")
    print(f"  Length: {len(address_str)} characters")
    
    # Get the scriptPubKey to see the underlying data
    script_pubkey = address_obj.to_script_pub_key()
    script_hex = script_pubkey.to_hex()
    script_bytes = bytes.fromhex(script_hex)
    
    if address_str[0] == '1' or address_str[0] == '3':
        # Base58Check encoded (P2PKH or P2SH)
        try:
            decoded = base58.b58decode(address_str)
            # Base58Check: version byte (1) + hash (20 bytes) + checksum (4 bytes) = 25 bytes
            print(f"  Format: Base58Check")
            print(f"  Decoded bytes: {len(decoded)} bytes")
            print(f"  Version byte: 0x{decoded[0]:02x}")
            print(f"  Hash160: {decoded[1:21].hex()} ({len(decoded[1:21])} bytes)")
            print(f"  Checksum: {decoded[21:].hex()} ({len(decoded[21:])} bytes)")
            print(f"  ScriptPubKey: {script_hex} ({len(script_bytes)} bytes)")
        except Exception as e:
            print(f"  Error decoding: {e}")
    
    elif address_str.startswith('bc1q'):
        # Bech32 encoded (P2WPKH)
        print(f"  Format: Bech32 (SegWit v0)")
        print(f"  ScriptPubKey: {script_hex} ({len(script_bytes)} bytes)")
        # P2WPKH script: OP_0 (0x00) + pushdata (0x14 = 20) + hash160 (20 bytes) = 22 bytes
        if len(script_bytes) == 22 and script_bytes[0] == 0x00 and script_bytes[1] == 0x14:
            print(f"  ✓ Correct format: OP_0 + pushdata(20) + 20-byte hash160")
            print(f"  Version: 0x00 (P2WPKH)")
            print(f"  Hash160: {script_bytes[2:].hex()} ({len(script_bytes[2:])} bytes)")
        else:
            print(f"  ⚠ Unexpected script format")
    
    elif address_str.startswith('bc1p'):
        # Bech32m encoded (P2TR)
        print(f"  Format: Bech32m (SegWit v1 / Taproot)")
        print(f"  ScriptPubKey: {script_hex} ({len(script_bytes)} bytes)")
        # P2TR script: OP_1 (0x51) + pushdata (0x20 = 32) + x-only pubkey (32 bytes) = 34 bytes
        if len(script_bytes) == 34 and script_bytes[0] == 0x51 and script_bytes[1] == 0x20:
            print(f"  ✓ Correct format: OP_1 + pushdata(32) + 32-byte x-only pubkey")
            print(f"  Version: 0x01 (P2TR)")
            print(f"  X-only pubkey: {script_bytes[2:].hex()} ({len(script_bytes[2:])} bytes)")
            print(f"  Note: Taproot addresses are longer because:")
            print(f"        - They use 32-byte x-only pubkeys (vs 20-byte hashes)")
            print(f"        - Bech32m encoding overhead")
            print(f"        - But provide better privacy and script flexibility")
        else:
            print(f"  ⚠ Unexpected script format")

# Setup mainnet
setup('mainnet')

# Generate a new Bitcoin private key
priv = PrivateKey()
pub = priv.get_public_key()

# Generate different address types
legacy_address = pub.get_address()
segwit_native = pub.get_segwit_address()
taproot_address = pub.get_taproot_address()

# For P2SH-P2WPKH
segwit_script = segwit_native.to_script_pub_key()
segwit_p2sh = P2shAddress.from_script(segwit_script)

print("=" * 70)
print("Bitcoin Address Format Verification")
print("=" * 70)

verify_address(legacy_address, legacy_address.to_string(), "Legacy (P2PKH)")
verify_address(segwit_native, segwit_native.to_string(), "SegWit Native (P2WPKH)")
verify_address(segwit_p2sh, segwit_p2sh.to_string(), "SegWit P2SH (P2SH-P2WPKH)")
verify_address(taproot_address, taproot_address.to_string(), "Taproot (P2TR)")

print("\n" + "=" * 70)
print("Summary:")
print("=" * 70)
print("P2PKH:      ~34 chars (Base58Check, 20-byte hash160)")
print("P2WPKH:     ~42-46 chars (Bech32, 20-byte hash160)")
print("P2SH-P2WPKH: ~34 chars (Base58Check, 20-byte script hash)")
print("P2TR:       ~58-62 chars (Bech32m, 32-byte x-only pubkey)")
print("\nTaproot addresses are longer because:")
print("  - They use 32-byte x-only public keys (not 20-byte hashes)")
print("  - Bech32m encoding is slightly less efficient than Base58Check")
print("  - But they provide better privacy and script flexibility")

While address encoding involves many subtle rules—version bytes, checksums, and different encodings (Base58Check, Bech32, Bech32m)—the more important point is understanding the overall concept:

👉 Addresses are designed for humans. They are just user-friendly representations of locking scripts (scriptPubKey), not essential components of the protocol itself.

Once you identify the prefix (1, 3, bc1q, bc1p), you know what type of script lies behind it. From a node's perspective, Bitcoin never stores addresses—only scripts.

In later chapters, we will focus on what truly matters: the actual scriptPubKey associated with each address type. That's where the real logic resides—and where Bitcoin Script and programmability begin. If you can predict the script behind an address, you can reason about how it can be spent.

## Derivation Model

Understanding the derivation relationship between keys and addresses is crucial. The diagram below captures the entire address derivation flow—from private key generation to the final on-chain script. While most wallet users only see addresses, developers need to trace the complete path to understand how Bitcoin enforces ownership and spending conditions.

```
Private Key (k)
    ↓ ECDSA multiplication
Public Key (x, y)
    ↓ SHA256 + RIPEMD160
Public Key Hash (20 bytes)
    ↓ Version + Checksum + Encoding
Address (Base58/Bech32)
  ↓ Decoded by wallet/node
ScriptPubKey (locking script on-chain)
```

**Security Properties:**

- **Forward derivation**: Each step is computationally easy
- **Reverse derivation**: Each step is computationally infeasible
- **Hash collision resistance**: Extremely low probability of different public keys producing the same address

## Practical Implications for Taproot

As we will see in later chapters, Taproot builds on these foundational concepts:

- **X-only public keys** reduce transaction size and enable key aggregation
- **Bech32m encoding** provides robust error detection for complex scripts
- **Unified address format** makes multi-signature and single-signature transactions indistinguishable

Understanding these encodings and key formats prepares us for Taproot's more sophisticated features, where multiple spending conditions can be combined into a single address format.

## Chapter Summary

This chapter established the cryptographic foundation for Bitcoin transactions:

- ✅ Private keys are 256-bit random numbers, encoded as WIF for usability
- ✅ Public keys are elliptic curve points, with compressed format as standard
- ✅ Addresses are encoded hashes of public keys, not public keys themselves
- ✅ Different address types use different encoding schemes: Base58Check, Bech32, and Bech32m
- ✅ Taproot introduces x-only public keys and Bech32m encoding for enhanced efficiency

All components introduced here—keys, hashes, encodings—are what Bitcoin Script ultimately operates on or verifies. In the next chapter, we will explore how these keys and addresses interact with Bitcoin Script—the programming language that defines spending conditions and enables Taproot's advanced features.